# Trail Recommendations

List everything that we need to do.

In [5]:
import Trailforks as tf
import TrailforksScraper as tfs
from bs4 import BeautifulSoup
import requests

## Getting data from Trailforks

- 'activitytype=6' for hiking
- Trailstyle = Popularity

Trailforks api documentation: https://www.trailforks.com/about/api/#!/trail

Trailforks popularity heatmap: https://www.trailforks.com/region/united-states/?activitytype=6&z=10.4&lat=48.30699&lon=-120.42421&trailstyle=popularity

Trailforks popularity scores (sorted in descending order): https://www.trailforks.com/region/united-states/trails/?sort=t.popularity_score&order=desc&difficulty=2,3,4,11,9,5,6,8&activitytype=6

In [6]:
# get popularity data
search = tf.trailforksSearch()
data = search.fetchPopularityData()
data.head()

url https://www.trailforks.com/region/united-states/trails/?sort=t.popularity_score&order=desc&difficulty=2,3,4,11,9,5,6,8&activitytype=6
Error occurred
[Errno Expecting value] <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>The Best Hiking Trails in United States | Trailforks</title>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<meta name="description" content="The best &amp; most popular hiking trails in United States. Trailforks ranks trails with a combination of user ratings and raw trail usage data from rides." />
<meta property="og:image" content="https://es.pinkbike.org/246/sprt/i/trailforks/trailforks_avatar.png"/>
<meta property="og:site_name" content="Trailforks" />
<meta property="fb:app_id" content="158879950980488" />
<meta property="fb:admins" content="517713210" />
<meta name="application-name" content="Trailforks" />
<meta name="apple-itunes-app" content="app-id=9879

AttributeError: 'trailforksSearch' object has no attribute 'data'

In [ ]:
data['trailtype']

0    1
1    1
2    2
Name: trailtype, dtype: object

In [ ]:
data['activitytype']

0    1
1    1
2    1
Name: activitytype, dtype: int64

In [ ]:
data['difficulty']

0    5
1    5
2    4
Name: difficulty, dtype: object

In [ ]:
search.fetchDataByTrailType("1")

,trailid,vid,title,rid,difficulty,activitytype,trailtype,biketype,physical_rating,ttfs,...,act_skialpine,act_skixc,act_skibc,act_horse,act_snowmobile,act_mototrials,latitude,longitude,staticmap_image1,activitytypes
0,3,2,Floppy Bunny,1,5,1,1,"1,2",2,"16,15",...,0,0,0,0,0,0,49.359050,-123.040270,https://ep1.pinkbike.org/trailstaticmap/0/3_2_...,"[1, 2, 6, 5]"
1,5,8,Pipeline,1,5,1,1,"1,2",2,"54,5,15,19",...,0,0,0,0,0,0,49.358990,-123.049780,https://ep1.pinkbike.org/trailstaticmap/0/5_8_...,"[1, 6, 5]"
2,6,5,Bobsled,1,4,1,2,"1,2,7",1,"1,54,17,104,21",...,0,0,0,0,0,0,49.358250,-123.040890,https://ep1.pinkbike.org/trailstaticmap/0/6_5_...,[1]


In [ ]:
search.fetchDataByTrailType("1")

In [ ]:
trailForksScrapper = tfs.trailforksScrapper()
trailForksScrapper.fetchTrailPopularityByName('rattlesnake-ledge-trail')

'\n100 in Trail Running '

In [5]:
north_carolina = trailForksScrapper.fetchTrailsByRegionAndPages('north-carolina',23)
washington = trailForksScrapper.fetchTrailsByRegionAndPages('washington',72)
print(len(north_carolina))
print(len(washington))
washington.head()

2300
7200


,Unnamed: 0,title,Unnamed: 2,riding area,rating,distance,descent,climb,popularity_score
0,NaN,unknown,NaN,Swan Creek Park,NaN,49 ft,-4 ft,NaN,0
1,NaN,unknown,NaN,Grand Ridge Park,NaN,442 ft,-32 ft,NaN,0
2,NaN,unknown,NaN,Tolt MacDonald Park,NaN,49 ft,NaN,4 ft,0
3,NaN,Iron Peak #1399,NaN,Teanaway,NaN,3 miles,-519 ft,"2,203 ft",0
4,NaN,Frog Trail,NaN,Pilchuck Tree Farm,NaN,485 ft,-9 ft,52 ft,0


In [6]:
north_carolina['title'] = north_carolina['title'].str.replace('\W', '-').str.lower()
washington['title'] = washington['title'].str.replace('\W', '-').str.lower()

/var/folders/7v/3vykhk5j10v0p4dt8gphqvyh0000gn/T/ipykernel_26644/3339383142.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  north_carolina['title'] = north_carolina['title'].str.replace('\W', '-').str.lower()
/var/folders/7v/3vykhk5j10v0p4dt8gphqvyh0000gn/T/ipykernel_26644/3339383142.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  washington['title'] = washington['title'].str.replace('\W', '-').str.lower()


In [7]:
washington.head(100)

,Unnamed: 0,title,Unnamed: 2,riding area,rating,distance,descent,climb,popularity_score
0,NaN,unknown,NaN,Swan Creek Park,NaN,49 ft,-4 ft,NaN,0
1,NaN,unknown,NaN,Grand Ridge Park,NaN,442 ft,-32 ft,NaN,0
2,NaN,unknown,NaN,Tolt MacDonald Park,NaN,49 ft,NaN,4 ft,0
3,NaN,iron-peak--1399,NaN,Teanaway,NaN,3 miles,-519 ft,"2,203 ft",0
4,NaN,frog-trail,NaN,Pilchuck Tree Farm,NaN,485 ft,-9 ft,52 ft,0
...,...,...,...,...,...,...,...,...,...
95,NaN,dike-road,NaN,Everett,NaN,"3,471 ft",-2 ft,4 ft,0
96,NaN,everett-junction-trail,NaN,Forest Park,NaN,"2,717 ft",-280 ft,21 ft,0
97,NaN,trail,NaN,Forest Park,NaN,676 ft,NaN,31 ft,0
98,NaN,kelley-hollow-road,NaN,Yakima,NaN,5 miles,-86 ft,"2,098 ft",0


In [8]:
for trail in washington.head()['title']:
    print(trailForksScrapper.fetchTrailStats(trail))
    

Error occurred
substring not found
Empty DataFrame
Columns: []
Index: []
Error occurred
substring not found
Empty DataFrame
Columns: []
Index: []
Error occurred
substring not found
Empty DataFrame
Columns: []
Index: []
Error occurred
list index out of range
Empty DataFrame
Columns: []
Index: []
                    ['Year'   'Check-Ins'                ['Month'
0                   ['2013'            10                     NaN
1                   ['2014'             1                     NaN
2                   ['2015'            20                     NaN
3                   ['2016'            36                     NaN
4                   ['2017'            38                     NaN
5                   ['2018'            88                     NaN
6                   ['2019'           153                     NaN
7                   ['2020'           114                     NaN
8                   ['2021'           151                     NaN
9                   ['2022'            82   

In [9]:
trailForksScrapper.fetchTrailStats('kelley-hollow-road')
trailForksScrapper.fetchTrailStats('rattlesnake-ledge-trail')

Error occurred
list index out of range


,['Year','Check-Ins',['Month'
0,['2014',1,NaN
1,['2017',2,NaN
2,['2018',1,NaN
3,['2020',4,NaN
4,['2021',228,NaN
5,['2022',251,NaN
0,NaN,20,['Jan'
1,NaN,25,['Feb'
2,NaN,22,['Mar'
3,NaN,54,['Apr'
